# Galderma - Accumulations
## Next year
* No RnD

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df_list = tabula.read_pdf("CH_EN_EFPIA_Report_20190620_FINAL.pdf", pages='all', lattice=True, multiple_tables = True)

In [3]:
#Take df with more than one column
df_list_new = []
for i in range(0, len(df_list)):
    if len(df_list[i].columns) > 1:
        df_list_new.append(df_list[i])
        
df = pd.concat(df_list_new)
df = df.reset_index(drop=True)

## Format Table

In [9]:
df_export = df.copy()
#Drop Columns
df_export = df_export.drop(df.columns[[0, 8]],axis=1)
df_export = df_export.drop(df.columns[10:],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of', na=False)]
rnd = df_export[df_export.type.str.contains('Transfers of Value re Research', na=False)]

#df_export = pd.concat([amount, recipients, rnd])
df_export = pd.concat([amount, recipients])

#Replace N/A-Strings
df_export.loc[df_export['donations_grants'] == 'N/A', 'donations_grants'] = np.NaN
df_export.loc[df_export['sponsorship'] == 'N/A', 'sponsorship'] = np.NaN

#Convert to Numbers
df_export = replace_in_number(df_export, ',', '')
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
#df_export.iloc[4,0] = "rnd"

#Shift RND from Sponsorship to total
#df_export.loc[df_export.type == 'rnd', 'total'] = df_export.sponsorship
#df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN
#Changed next year?
df_export = add_empty_accumulation(df_export, "rnd")

export_acumulations(df_export, 'galderma')


Total nicht Summe der Werte
saved


In [11]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)